In [ ]:
from sklearn.model_selection import KFold # import KFold
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) # create an array
y = np.array([1, 2, 3, 4]) # Create another array
kf = KFold(n_splits=2) # Define the split - into 2 folds 
kf.get_n_splits(X) # returns the number of splitting iterations in the cross-validator
print(kf) 
KFold(n_splits=2, random_state=None, shuffle=False)

In [ ]:
for train_index, test_index in kf.split(X):
 print(“TRAIN:”, train_index, “TEST:”, test_index)
 X_train, X_test = X[train_index], X[test_index]
 y_train, y_test = y[train_index], y[test_index]
('TRAIN:', array([2, 3]), 'TEST:', array([0, 1]))
('TRAIN:', array([0, 1]), 'TEST:', array([2, 3]))

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import numpy as np

import statsmodels.api as sm
from statsmodels.formula.api import ols

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
from math import sqrt



In [16]:
from haversine import haversine

In [17]:
df = pd.read_csv('house_data_cleaned.csv')

In [18]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,bed_bath_sum,renovation_yes,sqft_living_plus_lot,sqft_living_plus_lot_15
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,NaN,98178,47.5112,-122.257,1340,5650,4.00,0,6830,6990
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,1991.0,98125,47.7210,-122.319,1690,7639,5.25,1,9812,9329
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,NaN,98028,47.7379,-122.233,2720,8062,3.00,0,10770,10782
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,NaN,98136,47.5208,-122.393,1360,5000,7.00,0,6960,6360
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,NaN,98074,47.6168,-122.045,1800,7503,5.00,0,9760,9303


## lat,long tranformation into something useful
- after installing the haversine module, i want to explore how to use it to give us a distance from our building here in downtown. By doing so, it will allow us to check my hypothesis that homes closer to seattle are more expensive.

In [7]:
df.lat.head()

0    47.5112
1    47.7210
2    47.7379
3    47.5208
4    47.6168
Name: lat, dtype: float64

In [19]:

df['lat_long'] = tuple(zip(df.lat,df.long))


In [20]:
df['lat_long'].head()

0    (47.5112, -122.257)
1     (47.721, -122.319)
2    (47.7379, -122.233)
3    (47.5208, -122.393)
4    (47.6168, -122.045)
Name: lat_long, dtype: object

In [37]:
flatiron_latlong = [47.6092,-122.3363]
type(flatiron_latlong)

list

In [48]:
l = []
for i in df['lat_long']:
    l.append(haversine((flatiron_latlong),(i),unit = 'mi'))
l[:5]

[7.714867719017808,
 7.766476387009191,
 10.10794922047676,
 6.655340065489545,
 13.578402374482105]

In [49]:
df['distance_from_flatiron'] = pd.Series(l)


In [51]:
df.head()

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,lat,long,sqft_living15,sqft_lot15,bed_bath_sum,renovation_yes,sqft_living_plus_lot,sqft_living_plus_lot_15,lat_long,distance_from_flatiron
0,7129300520,2014-10-13,221900.0,3,1.00,1180,5650,1.0,0.0,0.0,...,47.5112,-122.257,1340,5650,4.00,0,6830,6990,"(47.5112, -122.257)",7.714868
1,6414100192,2014-12-09,538000.0,3,2.25,2570,7242,2.0,0.0,0.0,...,47.7210,-122.319,1690,7639,5.25,1,9812,9329,"(47.721, -122.319)",7.766476
2,5631500400,2015-02-25,180000.0,2,1.00,770,10000,1.0,0.0,0.0,...,47.7379,-122.233,2720,8062,3.00,0,10770,10782,"(47.7379, -122.233)",10.107949
3,2487200875,2014-12-09,604000.0,4,3.00,1960,5000,1.0,0.0,0.0,...,47.5208,-122.393,1360,5000,7.00,0,6960,6360,"(47.5208, -122.393)",6.655340
4,1954400510,2015-02-18,510000.0,3,2.00,1680,8080,1.0,0.0,0.0,...,47.6168,-122.045,1800,7503,5.00,0,9760,9303,"(47.6168, -122.045)",13.578402


In [52]:
df.to_csv('house_data_cleaned.csv')